In [ ]:
from simulator import *

import random
from utils import *
import numpy as np


def normalized_reciprocal(x):
    recip = 1/(x+1)  
    return recip / np.sum(recip)

def RandomReturn(stations_features):
    
    stations = []
    features = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        features.append(feature)
        
    return random.choice(stations)

def NearestReturn(stations_features):
    stations = []
    dists = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        dists.append(feature[2])
        
    min_index = np.argmin(dists)
    
    return stations[min_index]

def NearestPropReturn(stations_features):
    stations = []
    dists = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        dists.append(feature[2])
        
    dists = np.array(dists)
    
    prop = normalized_reciprocal(dists)
    
    return np.random.choice(stations,p=prop)

def MostNeededReturn(stations_features):
    stations = []
    truck_nums = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        truck_nums.append(feature[1])
        
    min_index = np.argmin(truck_nums)
    
    return stations[min_index]


def MostNeededProbReturn(stations_features):
    stations = []
    truck_nums = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        truck_nums.append(feature[1])
        
    truck_nums = np.array(truck_nums)
    
    prop = normalized_reciprocal(truck_nums)
    
    return np.random.choice(stations,p=prop)

    


def RunSimulator(simulator:Environment,return_method):
        
        simulator.log('-----------------------------------------------------')
        simulator.log('System Begins')
        
        last_time = None
        last_state = None
        last_action = None
        
        sample_sequence = []
        
        while(1):
        
            stations_features,current_time,regu_reward,current_truck  = env.step()
            
            if last_time:
                
                sample_sequence.append((last_time,last_state,last_action,regu_reward))
            
            
            if stations_features == None:
                break
                
            last_state = stations_features
            last_time = current_time
                
            action = return_method(stations_features)
            simulator.take_action(action,current_truck)
            
            last_action = action
            
        return sample_sequence

metric_dict = {}

from tqdm import tqdm

env = Environment(10,1,running_day=1,start_date=datetime.datetime(2024,5,2))

for func in [RandomReturn,NearestReturn,NearestPropReturn,MostNeededProbReturn,MostNeededReturn]:
    
    
    func_name = func.__name__
    
    print(func_name)
    
    metric_list = []
    
    test_num = 10
    
    if func == NearestReturn or func == MostNeededReturn:
        test_num = 1
    
    for i in tqdm(range(test_num)):
        
        env.reset()
        
        
        RunSimulator(env,func)
        
        metric_list.append(env.count_time_per_fang())
        
    
    m = np.mean(metric_list)
    print(m)
    metric_dict[func_name] = m
    
    



RandomReturn


100%|██████████| 10/10 [00:30<00:00,  3.02s/it]


7.1489141550863256
NearestReturn


100%|██████████| 1/1 [00:03<00:00,  3.25s/it]


11.394253358231431
NearestPropReturn


...

7.149636046441678
MostNeededPropReturn


100%|██████████| 10/10 [00:33<00:00,  3.31s/it]


6.878613968597726
MostNeededReturn


100%|██████████| 1/1 [00:03<00:00,  3.16s/it]

8.999819527161161


In [2]:
metric_dict

{'RandomReturn': np.float64(7.1489141550863256),
 'NearestReturn': np.float64(11.394253358231431),
 'NearestPropReturn': np.float64(7.149636046441678),
 'MostNeededPropReturn': np.float64(6.878613968597726),
 'MostNeededReturn': np.float64(8.999819527161161)}

In [2]:
metric_dict

{'RandomReturn': np.float64(5.703771375029281),
 'NearestReturn': np.float64(12.116186460529399),
 'NearestPropReturn': np.float64(5.903849457328024),
 'MostNeededPropReturn': np.float64(5.66532365112829),
 'MostNeededReturn': np.float64(7.364566252830484)}

5.1-5.2

{'RandomReturn': np.float64(7.12410465511383),
 'NearestReturn': np.float64(18.433530733101897),
 'NearestPropReturn': np.float64(7.480319401970777),
 'MostNeededPropReturn': np.float64(6.916561331974177)}

5.2
{'RandomReturn': np.float64(7.1489141550863256),
 'NearestReturn': np.float64(11.394253358231431),
 'NearestPropReturn': np.float64(7.149636046441678),
 'MostNeededPropReturn': np.float64(6.878613968597726),
 'MostNeededReturn': np.float64(8.999819527161161)}

5.1

{'RandomReturn': np.float64(5.703771375029281),
 'NearestReturn': np.float64(12.116186460529399),
 'NearestPropReturn': np.float64(5.903849457328024),
 'MostNeededPropReturn': np.float64(5.66532365112829),
 'MostNeededReturn': np.float64(7.364566252830484)}

In [2]:
import pandas as pd


# 读取 CSV 文件
df = pd.read_csv('shenzhen_orders_0324.csv', dtype=str)

# 提取 order_id 和 station_id，转换为字典
order_station_dict = df.set_index('order_id')['station_id'].to_dict()

# 打印结果
for k,v in order_station_dict.items():
    print(f"Order ID: {k}, Station ID: {v}")
    break

Order ID: 1649296900021358594, Station ID: 1631470956459929601


In [23]:
from component import *
import importlib
import simulator
importlib.reload(simulator)
from simulator import *

def return_dispatch_list(orders:List[Order]):
    
    dispatch_list :List[Dispatch]= []
    
    for order in orders:
        order_id = order.oid
        from_sid = order_station_dict[order_id]
        
        for _ in range(order.n_need):
            new_dispatch = Dispatch(order_id,order.pid,from_sid,order.plan_arrive_time-datetime.timedelta(hours=0),from_sid)
            dispatch_list.append(new_dispatch)
        
        
    dispatch_list.sort(key=lambda x: x.dispatch_time)
    
    return dispatch_list
    
    
def external_method_for_random_need(current_order:Order):
    
    dispatch_list = []
    
    oid = current_order.oid
    
    from_sid = order_station_dict[oid]
    
    truck_need = current_order.n_need
    
    for i in range(truck_need):
        d = Dispatch(oid,current_order.pid,from_sid,current_order.plan_arrive_time,from_sid)
        dispatch_list.append(d)    
    
    return dispatch_list

from simulator import *



env = Environment(50,4,running_day=1,start_date=datetime.datetime(2024,5,1))

dispatch_list_for_fixed_need = return_dispatch_list(env.unscheduled_orders.values())


env.continuous_running(dispatchs=dispatch_list_for_fixed_need,external_method=external_method_for_random_need)

print(env.penalty)
print(sum(env.penalty.values()))
print(env.revenue)
print(env.fail_dispatch)
print(env.cost)
print(env.fail_order)

Dispatchs can satisfy the orders, dispatchs and needs are equal.
{'waiting': 0, 'overtime': 17.5, 'discontinuity': 940000}
940017.5
2375275.0
91
83001.60000000015
0


In [14]:
env.interaction

{'680807003891019801': {'1631470956459929601': 18.5},
 '706166232260612241': {'738115065915781403': 21.5,
  '1631470956459929601': 12.5,
  '1663425866110771202': 15.5,
  '1673235371280293890': 14.5,
  '726724594072978268': 32.5,
  '1694634526754066434': 17.0},
 '833051041372312567': {'738115065915781403': 13.5,
  '1631470956459929601': 4.5,
  '1672793905165828098': 25.0,
  '1703960484185190401': 24.0,
  '1694634526754066434': 16.0},
 '680006810102620796': {'1631470956459929601': 11.0,
  '1673235371280293890': 13.0},
 '809026317189507013': {'1631470956459929601': 15.5,
  '1663425866110771202': 21.5,
  '1673235371280293890': 17.5,
  '1694634526754066434': 23.0,
  '755721905675469311': 26.0,
  '1782242566936788994': 21.5},
 '820716812386804551': {'754429282184478753': 8.0,
  '875336708021883376': 1.5,
  '1686320649152536578': 6.0,
  '755721905675469311': 7.0},
 '737416575015256065': {'1673235371280293890': 13.0},
 '781979182673667513': {'875336708021883376': 3.0,
  '1675689651580084225': 

In [41]:
env.penalty

{'waiting': 4119.0768, 'overtime': 41937.25, 'discontinuity': 0}